In [1]:
# !pip install torch
# # !pip install tqdm
# # !pip install scikit-learn


In [2]:
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoConfig
import evaluate
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from customModel import customBertForSequenceClassification
from CustomTraniner import CustomTrainer

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/leesk/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Data Process

In [4]:
data = pd.read_csv('./genre_seung.csv')
data.head()

,singer,song_name,index,emotion,caption,tempo,instruments,tempo(int),tempo(category),genre,sub_genre
0,"10,000 Maniacs",A Campfire Song,2,nostalgia,"The melody is mellow and soothing, with a gent...",Tempo_135.0,"['Program_-1', 'Program_16', 'Program_24', 'Pr...",135,Tempo_135,Rock,Alternative rock
1,101 Strings,Theme From The Godfather,3,nostalgia,Elegant and sweeping orchestral melody with a ...,Tempo_80.0,"['Program_-1', 'Program_0', 'Program_12', 'Pro...",80,Tempo_75,Pop,Pop
2,10cc,Dreadlock Holiday,4,excitement,"catchy and upbeat, featuring a memorable guita...",Tempo_103.0,"['Program_-1', 'Program_0']",103,Tempo_105,Rock,Reggae rock
3,10cc,I'm Not In Love,5,nostalgia,A catchy and memorable tune with a simple yet ...,Tempo_65.0,"['Program_-1', 'Program_0', 'Program_122', 'Pr...",65,Tempo_75,Rock,Soft rock
4,10cc,The Things We Do for Love,6,love,"The melody is catchy and memorable, with a ble...",Tempo_111.0,"['Program_-1', 'Program_0', 'Program_16', 'Pro...",111,Tempo_105,Rock,Soft rock


In [5]:
id2label_emotion = {k:l for k, l in enumerate(data.emotion.unique())}
label2id_emotion = {l:k for k, l in enumerate(data.emotion.unique())}
id2label_tempo = {k:l for k, l in enumerate(data['tempo(category)'].unique())}
label2id_tempo = {l:k for k, l in enumerate(data['tempo(category)'].unique())}
id2label_genre = {k:l for k, l in enumerate(data['genre'].unique())}
label2id_genre = {l:k for k, l in enumerate(data['genre'].unique())}

In [6]:
train_data, valid_data = train_test_split(data, stratify=data['emotion'],test_size= 0.1, random_state=42)

In [7]:
class customDataset_before():
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.dataset = []
        datas = []
        self.labels = []
        for idx, df in tqdm(data.iterrows()):
            label = [0. for _ in range(num_labels)]
            datas.append(df.caption)
            label[label2id[df.emotion]] = 1.
            label[label2id[df['tempo(category)']]] = 1.
            self.labels.append(label)
        self.dataset =  tokenizer(datas,padding=True, truncation=True,max_length=512 ,return_tensors="pt").to('cuda')
        self.labels= torch.tensor(self.labels)
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.dataset.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item
        # return {'input_ids' : self.dataset[idx]['input_ids'],
        #         'attention_mask' : self.dataset[idx]['attention_mask'],
        #         'labels' : self.labels[idx]
        #         }

In [8]:
class customDataset_after():
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.dataset = []
        datas = []
        self.labels1 = []
        self.labels2 = []
        self.labels3 = []
        for idx, df in tqdm(data.iterrows()):
            label1 = [0. for _ in range(len(id2label_emotion))]
            label2 = [0. for _ in range(len(id2label_tempo))]
            label3 = [0. for _ in range(len(id2label_genre))]
            datas.append(df.caption)
            label1[label2id_emotion[df.emotion]] = 1.
            label2[label2id_tempo[df['tempo(category)']]] = 1.
            label3[label2id_genre[df['genre']]] = 1.
            self.labels1.append(label1)
            self.labels2.append(label2)
            self.labels3.append(label3)
        self.dataset =  tokenizer(datas,padding=True, truncation=True,max_length=512 ,return_tensors="pt").to('cuda')
        self.labels1= torch.tensor(self.labels1)
        self.labels2= torch.tensor(self.labels2)
        self.labels3= torch.tensor(self.labels3)
    def __len__(self):
        return len(self.labels1)
    
    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.dataset.items()}
        item['labels1'] = self.labels1[idx].clone().detach()
        item['labels2'] = self.labels2[idx].clone().detach()
        item['labels3'] = self.labels3[idx].clone().detach()
        return item

In [9]:
# BASE_MODEL = 'bert-base-uncased'

# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=num_labels,
#            id2label=id2label, label2id=label2id, problem_type = "multi_label_classification").to('cuda')


In [10]:
BASE_MODEL = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
config = AutoConfig.from_pretrained(BASE_MODEL)
config.num_labels1 = len(id2label_emotion)
config.num_labels2 = len(id2label_tempo)
config.num_labels3 = len(id2label_genre)
model = customBertForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
# custom = customBertForSequenceClassification(config, num_labels1 = len(id2label_emotion), 
#                                                             num_labels2 = len(id2label_tempo), num_labels3 = len(id2label_genre))
# model = customBertForSequenceClassification.from_pretrained(config)

Some weights of customBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier1.bias', 'classifier1.weight', 'classifier2.bias', 'classifier2.weight', 'classifier3.bias', 'classifier3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
dataset_train = customDataset_after(train_data, tokenizer =tokenizer)
dataset_valid = customDataset_after(valid_data, tokenizer =tokenizer)


9045it [00:00, 10190.91it/s]
1006it [00:00, 12364.89it/s]


In [12]:
GLOBAL_SCORE_INDICES = range(0,17)
CAUSE_INDICES = range(17, 25)
def get_preds_from_logits(logits):
    ret = np.zeros(logits.shape)
    
    # The first 5 columns (GLOBAL_SCORE_INDICES) are for global scores. They should be handled with a multiclass approach
    # i.e. we fill 1 to the class with highest probability, and 0 into the other columns
    best_class = np.argmax(logits, axis=1)
    ret[list(range(len(ret))), best_class] = 1
    # The other columns are for causes and emotions. They should be handled with multilabel approach.
    # i.e. we fill 1 to every class whose score is higher than some threshold
    # In this example, we choose that threshold = 0
    
    return ret

In [13]:
# clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

# def sigmoid(x):
#    return 1/(1 + np.exp(-x))

# def compute_metrics(eval_pred):

#    predictions, labels = eval_pred
#    predictions = sigmoid(predictions)
#    predictions = (predictions > 0.5).astype(int).reshape(-1)
#    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    final_metrics = {}
    
    # Deduce predictions from logits
    predictions_emotion = get_preds_from_logits(logits[0])
    predictions_tempo = get_preds_from_logits(logits[1])
    predictions_genre = get_preds_from_logits(logits[2])
    
    # Get f1 metrics for global scoring. Notice that f1_micro = accuracy
    final_metrics["f1_emotion"] = f1_score(labels[0], predictions_emotion, average="micro")
    
    # Get f1 metrics for causes
    final_metrics["f1_tempo"] = f1_score(labels[1], predictions_tempo, average="micro")
    

    # The global f1_metrics
    final_metrics["f1_genre"] = f1_score(labels[2], predictions_genre, average="micro")
    
    # Classification report
    # print("Classification report for global scores: ")
    # print(classification_report(labels[:, GLOBAL_SCORE_INDICES], predictions[:, GLOBAL_SCORE_INDICES], zero_division=0))
    # print("Classification report for causes: ")
    # print(classification_report(labels[:, CAUSE_INDICES], predictions[:, CAUSE_INDICES], zero_division=0))
    return final_metrics

In [15]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
class MultiTaskClassificationTrainer(Trainer):
    def __init__(self, group_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.group_weights = group_weights
        
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        
        global_score_loss = torch.nn.functional.cross_entropy(logits[:, GLOBAL_SCORE_INDICES], labels[:, GLOBAL_SCORE_INDICES])
        cause_loss = torch.nn.functional.binary_cross_entropy_with_logits(logits[:, CAUSE_INDICES], labels[:, CAUSE_INDICES])
        
        loss = self.group_weights[0] * global_score_loss +self.group_weights[1] * cause_loss
        return (loss, outputs) if return_outputs else loss

In [17]:
training_args = TrainingArguments(

   output_dir="my_awesome_model",
   learning_rate=2e-5,
   per_device_train_batch_size=32,
   per_device_eval_batch_size=32,
   num_train_epochs=10,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
   save_total_limit = 3,
)

trainer = CustomTrainer(

   model=model,
   args=training_args,
   train_dataset=dataset_train,
   eval_dataset=dataset_valid,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


# trainer = Trainer(

#    model=model,
#    args=training_args,
#    train_dataset=dataset_train,
#    eval_dataset=dataset_valid,
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics,
# )

# trainer = MultiTaskClassificationTrainer(

#    model=model,
#    args=training_args,
#    train_dataset=dataset_train,
#    eval_dataset=dataset_valid,
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics,
#    group_weights=(0.7, 4)
# )



trainer.train()


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


label 1 번째
label 2 번째
label 3 번째


Epoch,Training Loss,Validation Loss


label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째
label 2 번째
label 3 번째
label 1 번째

KeyboardInterrupt: 